In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

import numpy as np
import pandas as pd
import json
from scipy.special import erf

In [ ]:
def twoexp(time, theta):
    """The model method contains the actual function definition.
    Returns a numpy-array of size len(self.times)
    Parameters:
    event_time = start time of word relative to start time of time series
    scale = horizontal scale parameter to stretch/compress word
    skew = skewness parameter: how much faster is the rise than the decay?
    """
    t0 = theta[0]
    amp = theta[1]
    scale = theta[2]
    skew = theta[3]

    t = (time - t0) / scale
    y = np.zeros_like(t)
    y[t <= 0] = np.exp(t[t <= 0])
    y[t > 0] = np.exp(-t[t > 0] / skew)

    y = np.array(y) * amp

    return y

def model(time, theta):
    t0 = theta[0]
    amp = theta[1]
    rise = theta[2]
    skew = theta[3]

    t = (time - t0)
    erf_inner = skew * (t/rise)/np.sqrt(2)
    cdf = 0.5 * (1+ erf(erf_inner))
    pdf_fac = 1.0 / (rise * np.pow(2*np.pi, 0.5))
    pdf = pdf_fac * np.exp(-np.pow(t,2)) / (2*np.pow(rise,2))
    return amp * 2 * pdf * cdf

In [ ]:
import dnest4
dnest4.postprocess()

#### Loading the data saved in the `.json` file

In [ ]:
data = json.load(open("temp.json"))
time = np.array(data["times"])
ytrue = np.array(data["profile"])
yerr = np.array(data["std"])

In [ ]:
data['eid']

In [ ]:
# plt.errorbar(time, ytrue, yerr=yerr, fmt=".", label="Profile with errorbars")
plt.fill_between(time, ytrue-yerr, ytrue+yerr, alpha=0.5, color='blue')
plt.plot(time, ytrue, c="black", label="Profile")
plt.xlabel("Time (s)")
plt.ylabel("Time profile")
plt.legend()
plt.title("Time profile of burst loaded in temp.json")
plt.show()

#### Loading the posterior sample

In [ ]:
sample = np.loadtxt("100b_10k_results/posterior_sample.txt", skiprows=2)
print(f"Number of sample saved in 'posterior_sample.txt': {sample.shape[0]}.")

In [ ]:
background = sample[:, 0]

# Number of dimensions for a single component
burst_dims = list(set(sample[:, 1]))[0]

# Maximum number of components allowed for the model
max_components = int(list(set(sample[:, 2]))[0])

# Now loading back the hyper parameters:
# hyper-parameter (mean) of the exponential distribution used
# as prior for the spike amplitudes
# NOTE: IN LINEAR SPACE, NOT LOG
hyper_mean_amplitude = sample[:, 3]

# hyper-parameter (mean) for the exponential distribution used
# as prior for the spike rise time
# NOTE: IN LINEAR SPACE, NOT LOG
hyper_mean_risetime = sample[:, 4]

# hyper-parameters for the lower and upper limits of the uniform
# distribution osed as a prior for the skew
hyper_lowerlimit_skew = sample[:, 7]
hyper_upperlimit_skew = sample[:, 8]

# Distribution of the number of components:
nbursts = sample[:, 9]

# individual burst parameters for all 100 components
npos = sample[:, 10:10+max_components]  # peak position for all burst components
amp = sample[:, 10+max_components:10+max_components*2]  # amplitude for all burst components
scale = sample[:, 10+max_components*2:10+max_components*3]  # rise time for all burst components
skew = sample[:, 10+max_components*3:10+max_components*4]  # skewness parameter for all burst components

# put all of the parameters together
pars_all = np.array([npos, amp, scale, skew]).T

# # all of the mean models
ymodel_all = sample[:, -len(time) :]  # model flux

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

axes = np.hstack(axes)

axes[0].hist(background, bins=20)
axes[0].set_title("background parameter")

axes[1].hist(hyper_mean_amplitude, bins=20)
axes[1].set_title("mean amplitude hyperparameter")

axes[2].hist(hyper_mean_risetime, bins=20)
axes[2].set_title("mean rise time hyperparameter")

unique_values, counts = np.unique(nbursts, return_counts=True)

# axes[3].hist(nbursts, bins=max_components-1)
axes[3].bar(unique_values, counts)
axes[3].set_title("number of components")
plt.show()

In [ ]:
plt.bar(unique_values, counts)
plt.title("Distribution of number of components")
plt.xlabel("Number of components")
plt.ylabel("Counts")
plt.savefig("hist.jpg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

ax.plot(time, ytrue)
for i in range(3):
    ax.plot(time, ymodel_all[i, :], label=f'Model {i}  N:{int(nbursts[i])}')
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
yfinal = ymodel_all.mean(0)
ax.plot(time, ytrue)
ax.plot(time, yfinal)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
yfinal2 = np.median(sample, axis=0)

ax.plot(time, ytrue)
# ax.plot(time, yfinal2)
# individual burst parameters for all 100 components
m_npos = yfinal2[10:10+max_components]  # peak position for all burst components
m_amp = yfinal2[10+max_components:10+max_components*2]  # amplitude for all burst components
m_scale = yfinal2[10+max_components*2:10+max_components*3]  # rise time for all burst components
m_skew = yfinal2[10+max_components*3:10+max_components*4]  # skewness parameter for all burst components

for pos, amp, scale, skew in zip(m_npos, m_amp, m_scale, m_skew):
    if pos != 0:
        # print(pos, amp, scale, skew)
        ax.plot(time, yfinal2[0]+twoexp(time, [pos, np.pow(1e20, amp), scale, skew]))

ax.plot(time, yfinal2[-len(time):])

In [ ]:
from findpeaks import findpeaks
fp = findpeaks(method='topology', lookahead=1)
results = fp.fit(yfinal)
# Plot
fp.plot1d()

In [ ]:
df = results['df']

In [ ]:
filtered = df[df['peak']][~df['valley']][df['score'] >= 0.25].sort_values(by='rank')

In [ ]:
filtered

In [ ]:
plt.plot(time, ytrue, label="Data")
plt.plot(time, yfinal, label="Model")
for p in filtered['x'].tolist():
    plt.axvline(time[p], c='grey', ls='--')
plt.xlabel("Time (s)")
plt.ylabel("SNR")
plt.title(f"Time profile")
plt.legend()
# plt.savefig("test.jpg")

In [ ]:
writer = ocf.npz_writer()
writer.set_data(r.data_full, r.times, r.freqs, r.res_time, r.res_freq)
writer.update_burst_parameters(
    arrival_time=r.times[filtered['x']]
)
# writer.burst_parameters
# writer.save("/arc/home/clegue/auto-bb-fitburst/npz_edited/245936178_copy.npz")

In [ ]:
r = ocf.npz_reader("temp.npz")
sns.set_style("white")
p = ocf.waterfall_axes()
p.plot(r)
# p.save("image.jpg")